<a href="https://colab.research.google.com/github/sanjit1995/TelstraNetworkDisruptions_TechGig/blob/master/TelstraNetworkDisruptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/TelstraNWDisruptions/train.csv")
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/TelstraNWDisruptions/test.csv")
severity = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/TelstraNWDisruptions/severity-type.csv")
resource = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/TelstraNWDisruptions/resource-type.csv")
log = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/TelstraNWDisruptions/log-feature.csv")
event = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/TelstraNWDisruptions/event-type.csv")

In [3]:
resource.count()

id               21076
resource_type    21076
dtype: int64

In [0]:
np.unique(resource['id']).size

18552

In [0]:
np.unique(severity['id']).size

18552

In [0]:
ids = resource['id']
resource[ids.isin(ids[ids.duplicated()])]

In [0]:
resource.info()

In [0]:
resource.pivot(index='id',columns='resource_type', values='resource_type')

In [0]:
resource_df = resource.copy()
resource_df = pd.get_dummies(resource_df['resource_type'], drop_first=False)
resource_df = pd.concat([resource['id'],resource_df], axis=1)
resource_df = resource_df.groupby(['id'], as_index=False, sort=False).sum()
resource_df

In [0]:
resource_df

In [0]:
new_df.loc[new_df['id'] == 9320]

In [0]:
event_df = event.copy()
event_df = pd.get_dummies(event_df['event_type'], drop_first=False)
event_df = pd.concat([event['id'],event_df], axis=1)
event_df = event_df.groupby(['id'], as_index=False, sort=False).sum()
event_df.id = event_df.id.astype(int)

In [0]:
diff_df = resource_df['id'] - event_df['id']
diff_df

In [0]:
event_df.loc[0]['id']

In [0]:
for i in range(0, len(resource_df)):
  if (resource_df.loc[i]['id'] - event_df.loc[i]['id']) != 0:
      print("What!!!!!!!!!!")

In [0]:
severity_df = severity.copy()
severity_df = pd.get_dummies(severity_df['severity_type'], drop_first=False)
severity_df = pd.concat([severity['id'],severity_df], axis=1)
severity_df = severity_df.groupby(['id'], as_index=False, sort=False).sum()
severity_df

In [8]:
print(resource_df.shape, event_df.shape, severity_df.shape)

(18552, 11) (18552, 54) (18552, 6)


In [0]:
event_df.loc[:,event_df.columns!='id']

In [0]:
extras_df = pd.concat([resource_df,event_df.loc[:,event_df.columns!='id'],severity_df.loc[:,severity_df.columns!='id']],axis=1)
extras_df

In [19]:
extras_df.shape

(18552, 69)

In [0]:
ids = extras_df['id']
extras_df[ids.isin(ids[ids.duplicated()])]

In [0]:
extras_df.loc[extras_df['id'] == 9320]

In [22]:
train.shape

(5999, 3)

In [0]:
train.loc[:,train.columns=='id']

In [0]:
train_data = pd.merge(train,extras_df)

In [0]:
train_data.head(3)

In [0]:
test.isnull().values.any()

In [0]:
x_train = train_data.drop(['fault_severity'], axis=1)
y_train = train_data['fault_severity']

In [39]:
x_train.shape

(5999, 70)

In [0]:
test = pd.merge(test,extras_df)

In [44]:
test.shape

(1382, 70)

In [0]:
train_data.corr()

In [0]:
x_train.to_csv('/content/drive/My Drive/Colab Notebooks/data/TelstraNWDisruptions/x_train.csv')
y_train.to_csv('/content/drive/My Drive/Colab Notebooks/data/TelstraNWDisruptions/y_train.csv')
test.to_csv('/content/drive/My Drive/Colab Notebooks/data/TelstraNWDisruptions/test.csv')

In [0]:
x_train['location'] = x_train['location'].map(lambda x: x.lstrip('location '))

In [0]:
test['location'] = test['location'].map(lambda x: x.lstrip('location '))

In [0]:
x_train.head(5)

In [57]:
print(x_train.shape, y_train.shape, test.shape)

(5999, 70) (5999,) (1382, 70)


In [0]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(verbose=2, n_estimators=100)

In [0]:
classifier.fit(x_train,y_train)

In [62]:
y_pred = classifier.predict(test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [0]:
unique, counts = np.unique(y_pred, return_counts=True)

print(np.asarray((unique, counts)).T)

In [0]:
pred = pd.DataFrame(y_pred)
sub_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/TelstraNWDisruptions/test.csv")
predFile = pd.concat([sub_df['id'],pred], axis=1)
predFile.columns = ['id', 'Prediction']

In [85]:
predFile['Prediction'].value_counts()

0    964
1    296
2    122
Name: Prediction, dtype: int64

In [0]:
test.loc[test['id'] == 11066]

In [0]:
predFile

In [0]:
pred_dummies = pd.get_dummies(predFile["Prediction"],prefix='predict',drop_first=False)

In [0]:
pred_dummies

In [0]:
predFinal = pd.concat([sub_df['id'],pred_dummies], axis=1)

In [0]:
predFinal

In [0]:
predFinal.to_csv('predFinal.csv', index=False)